In [ ]:
!pip install -U awswrangler

In [ ]:
import awswrangler as wr
import boto3
from datetime import datetime, timedelta
import logging
import pandas as pd
from pathlib import Path 
import numpy as np
from typing import List

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def generate_delta_reports(input_list: List[str], target_database: str, target_workgroup: str, output_s3_path: str) -> None:
    today = datetime.today()
    date_formatted = today.strftime("%Y%m%d")

    for item in input_list:
        path = Path(item)
        if path.is_dir():
            folder_name = path.name.replace("_delta_validation_sql_queries", "") 
            for sql_file_path in path.glob("*.sql"):
                if sql_file_path.is_file():
                    with sql_file_path.open('r') as f:
                        query_str = f.read()
                    df = wr.athena.read_sql_query(sql=query_str, database=target_database, workgroup=target_workgroup)
                    
                    if sql_file_path.name == 'xo_channels_brand_counts_diffs.sql':
                        first_row = df.index[0]
                        df.loc[df.index != first_row, ['old_total_record_count', 'new_total_record_count']] = np.nan
                    
                    output_path = f"{output_s3_path}{folder_name}/{date_formatted}/{sql_file_path.stem}.csv"
                    logger.info(f"Loading {sql_file_path.name} into {output_path}")
                    wr.s3.to_csv(df, output_path, index=False)
                else:
                    raise FileNotFoundError(f"File {sql_file_path} not found.")
        else:
            raise NotADirectoryError(f"{item} is not a directory.")

In [ ]:
if __name__ == "__main__":
    input_list = [
        '/root/MARS_ETL_PROCESSES/sql/delta_report_queries/xo_delta_validation_sql_queries/',
        '/root/MARS_ETL_PROCESSES/sql/delta_report_queries/speaker_delta_validation_sql_queries/',
        '/root/MARS_ETL_PROCESSES/sql/delta_report_queries/sales_delta_validation_sql_queries/',
        '/root/MARS_ETL_PROCESSES/sql/delta_report_queries/tam_delta_validation_sql_queries/'
    ]
    target_database = 'sandbox_measurement_analyst'
    target_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'
    output_s3_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_data_quality_reports/mars_delta_diff_report/'
    generate_delta_reports(input_list, target_database, target_workgroup, output_s3_path)